# Import Library

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Preprocessing

In [8]:
df = pd.read_csv('dataset.csv')
df_melab = df[(df['station'] == 'MELAB') & (df['class'] != 'GENDOL')]
df_melab

,mean,standard_deviation,median,min_value,max_value,range,interquartile_range,q1,q3,kurtosis,skewness,station,channel,class
0,-10087.594937,1095.953766,-10445.0,-14409,-7321,7088,1950.00,-10895.0,-8945.00,-0.844598,0.024230,MELAB,HHN,BEBENG
1,-8315.313791,1203.245256,-8298.0,-14163,-2830,11333,771.00,-8675.0,-7904.00,6.035557,-0.319061,MELAB,HHZ,BEBENG
2,-11952.875455,1373.377233,-11634.5,-18896,-7042,11854,1283.25,-12461.0,-11177.75,3.624293,-1.143653,MELAB,HHE,BEBENG
6,-12356.716855,1779.579821,-12849.0,-16766,-5367,11399,2072.00,-13566.0,-11494.00,1.122037,1.062310,MELAB,HHE,BEBENG
7,-10712.443704,1240.537369,-10833.0,-15870,-4034,11836,1215.00,-11395.0,-10180.00,3.125415,0.593652,MELAB,HHN,BEBENG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,-10710.962025,3899.671214,-10723.0,-26459,9529,35988,1863.00,-11765.0,-9902.00,5.332670,0.333064,MELAB,HHN,BOYONG
776,-7124.676216,2258.803599,-7119.0,-17085,2943,20028,1543.00,-7881.0,-6338.00,4.557207,-0.266929,MELAB,HHZ,BOYONG
780,-13135.842105,1953.064907,-13229.0,-20892,-2009,18883,2533.00,-14524.0,-11991.00,3.673440,0.885243,MELAB,HHE,BOYONG
781,-10648.238508,1532.024860,-10645.0,-18511,-2379,16132,1287.00,-11300.0,-10013.00,5.490923,0.216393,MELAB,HHN,BOYONG


# Data Cleaning

In [10]:
df_melab.isnull().sum()

mean                   0
standard_deviation     0
median                 0
min_value              0
max_value              0
range                  0
interquartile_range    0
q1                     0
q3                     0
kurtosis               0
skewness               0
station                0
channel                0
class                  0
dtype: int64

# Persebaran Kelas

In [11]:
count_boyong = len(df_melab[df_melab['class'] == 'BOYONG'])
count_bebeng = len(df_melab[df_melab['class'] == 'BEBENG'])


print(count_boyong)
print(count_bebeng)

21
372


In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

# Assuming merged_df is your DataFrame
# Separate features and labels
X = df_melab.drop(['class','station', 'channel'], axis=1)
y = df_melab['class']

# Label encoding for the target label
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.1, stratify=y_encoded, random_state=42)

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [13]:
print("========================SEBELUM SMOTE========================== \n")

# Mendapatkan jumlah data untuk setiap kelas dalam y_train dan y_test
train_class_counts = pd.Series(y_train).value_counts()
test_class_counts = pd.Series(y_test).value_counts()

# Menampilkan jumlah data untuk setiap kelas
print("Data Latih:")
for class_label, count in train_class_counts.items():
    print(f"{label_encoder.inverse_transform([class_label])[0]} = {count} data")

print("\nData Uji:")
for class_label, count in test_class_counts.items():
    print(f"{label_encoder.inverse_transform([class_label])[0]} = {count} data")
    
print("========================SESUDAH SMOTE========================== \n")
# Mendapatkan jumlah data untuk setiap kelas dalam y_train dan y_test
train_class_counts = pd.Series(y_train_smote).value_counts()
test_class_counts = pd.Series(y_test).value_counts()

print("Data Latih:")
for class_label, count in train_class_counts.items():
    print(f"{label_encoder.inverse_transform([class_label])[0]} = {count} data")

print("\nData Uji:")
for class_label, count in test_class_counts.items():
    print(f"{label_encoder.inverse_transform([class_label])[0]} = {count} data")  
    


========================SEBELUM SMOTE========================== 

Data Latih:
BEBENG = 334 data
BOYONG = 19 data

Data Uji:
BEBENG = 38 data
BOYONG = 2 data
========================SESUDAH SMOTE========================== 

Data Latih:
BEBENG = 334 data
BOYONG = 334 data

Data Uji:
BEBENG = 38 data
BOYONG = 2 data


In [14]:
class_names = df_melab['class'].unique()

In [16]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Inisialisasi model KNN
knn_model = KNeighborsClassifier()

# Tentukan rentang nilai K yang ingin dieksplorasi
param_dist = {
    'n_neighbors': range(1, 37),
    'leaf_size': range(20, 60),
    'p' : [2]
}
  # Contoh: kita ingin mencoba nilai K dari 1 hingga 20

# Inisialisasi objek RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=knn_model, param_distributions=param_dist, n_iter=100, cv=3, scoring='accuracy', random_state=42)

# Latih model menggunakan Random Search
random_search.fit(X_train_smote, y_train_smote)

# Tampilkan hasil pencarian parameter
print("Best parameter (K):", random_search.best_params_)
print("Best cross-validation score:", random_search.best_score_)

# Evaluasi model terbaik pada data uji
best_knn_model = random_search.best_estimator_
y_pred = best_knn_model.predict(X_test)

# Tampilkan laporan klasifikasi
print(classification_report(y_test, y_pred, target_names=class_names))


Best parameter (K): {'p': 2, 'n_neighbors': 2, 'leaf_size': 42}
Best cross-validation score: 0.7904092433240416
              precision    recall  f1-score   support

      BEBENG       0.96      0.68      0.80        38
      BOYONG       0.08      0.50      0.13         2

    accuracy                           0.68        40
   macro avg       0.52      0.59      0.47        40
weighted avg       0.92      0.68      0.77        40



In [17]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Inisialisasi model KNN
knn_model = KNeighborsClassifier()

# Tentukan rentang nilai K yang ingin dieksplorasi
param_dist = {
    'n_neighbors': range(1, 37),
    'leaf_size': range(20, 60),
    'p' : [2]
}
  # Contoh: kita ingin mencoba nilai K dari 1 hingga 20

# Inisialisasi objek RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=knn_model, param_distributions=param_dist, n_iter=100, cv=5, scoring='accuracy', random_state=42)

# Latih model menggunakan Random Search
random_search.fit(X_train_smote, y_train_smote)

# Tampilkan hasil pencarian parameter
print("Best parameter (K):", random_search.best_params_)
print("Best cross-validation score:", random_search.best_score_)

# Evaluasi model terbaik pada data uji
best_knn_model = random_search.best_estimator_
y_pred = best_knn_model.predict(X_test)

# Tampilkan laporan klasifikasi
print(classification_report(y_test, y_pred, target_names=class_names))


Best parameter (K): {'p': 2, 'n_neighbors': 2, 'leaf_size': 42}
Best cross-validation score: 0.8009089888901357
              precision    recall  f1-score   support

      BEBENG       0.96      0.68      0.80        38
      BOYONG       0.08      0.50      0.13         2

    accuracy                           0.68        40
   macro avg       0.52      0.59      0.47        40
weighted avg       0.92      0.68      0.77        40



In [18]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Inisialisasi model KNN
knn_model = KNeighborsClassifier()

# Tentukan rentang nilai K yang ingin dieksplorasi
param_dist = {
    'n_neighbors': range(1, 37),
    'leaf_size': range(20, 60),
    'p' : [2]
}
  # Contoh: kita ingin mencoba nilai K dari 1 hingga 20

# Inisialisasi objek RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=knn_model, param_distributions=param_dist, n_iter=100, cv=10, scoring='accuracy', random_state=42)

# Latih model menggunakan Random Search
random_search.fit(X_train_smote, y_train_smote)

# Tampilkan hasil pencarian parameter
print("Best parameter (K):", random_search.best_params_)
print("Best cross-validation score:", random_search.best_score_)

# Evaluasi model terbaik pada data uji
best_knn_model = random_search.best_estimator_
y_pred = best_knn_model.predict(X_test)

# Tampilkan laporan klasifikasi
print(classification_report(y_test, y_pred, target_names=class_names))


Best parameter (K): {'p': 2, 'n_neighbors': 2, 'leaf_size': 31}
Best cross-validation score: 0.8024197195838987
              precision    recall  f1-score   support

      BEBENG       0.96      0.68      0.80        38
      BOYONG       0.08      0.50      0.13         2

    accuracy                           0.68        40
   macro avg       0.52      0.59      0.47        40
weighted avg       0.92      0.68      0.77        40

